使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第四章第七節 温水暖房）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import nbimporter
from s03_01_hc_load_general import get_A_HCZ_i

## 付録H　温水暖房用熱源機の最大能力 / Appendix H Maximum capacity of heat source equipment for hot water heating 

### H.1 記号及び単位 / Symbol and unit

省略

ommision

### H.2 温水暖房用熱源機の最大能力 / Maximum capacity of heat source equipment for hot water heating

$$\large
q_{max,hs} = q_{rq,H} \times \sum_{i} A_{HCZ,i} \times f_{cT} \times f_{cI}
$$

$q_{max,hs}$：温水暖房用熱源機の最大能力(W) / Maximum capacity of heat source equipment for hot water heating  
$q_{rq,H}$：単位面積当たりの必要暖房能力(W/m<sup>2</sup>) / Required heating capacity per unit area  
$A_{HCZ,i}$：暖冷房区画$i$の床面積(m<sup>2</sup>) / Floor area of heating / cooling section $ i $  
$f_{cT}$：外気温度能力補正係数 / Outside air temperature ability correction factor  
$f_{cI}$：間歇運転能力補正係数 / Intermittent driving ability correction factor

In [ ]:
def calc_q_max_hs(region, A_A, A_MR, A_OR, mode_MR, mode_OR, has_MR_hwh, has_OR_hwh):
    """ 温水暖房用熱源機の最大能力 (1)

    :param region: 省エネルギー地域区分
    :type region: int
    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param A_MR: 主たる居室の床面積 (m2)
    :type A_MR: float
    :param A_OR: その他の居室の床面積 (m2)
    :type A_OR: float
    :param mode_MR: 主たる居室の運転モード 'い', 'ろ', 'は'
    :type mode_MR: str
    :param mode_OR: その他の居室の運転モード 'い', 'ろ', 'は'
    :type mode_OR: str
    :param has_MR_hwh: 温水暖房の放熱器を主たる居室に設置する場合はTrue
    :type has_MR_hwh: bool
    :param has_MR_hwh: 温水暖房の放熱器を主たる居室に設置する場合はTrue
    :type has_OR_hwh: bool
    :return: 温水暖房用熱源機の最大能力 (W)
    :rtype: float
    """

    # 単位面積当たりの必要暖房能力
    q_rq_H = get_q_rq_H(region, has_MR_hwh, has_OR_hwh)

    # 外気温度補正係数
    f_cT = get_f_cT()

    # 間歇運転能力補正係数
    f_cI = get_f_cI(mode_MR, mode_OR, has_MR_hwh, has_OR_hwh)

    # 暖冷房区画の床面積(温水暖房により暖 房される暖冷房区画のみを積算する)
    A_HCZ_hs = 0
    if has_MR_hwh:
        A_HCZ_hs = get_A_HCZ_i(1, A_A, A_MR, A_OR)
    if has_OR_hwh:
        A_HCZ_hs = A_HCZ_hs + sum([get_A_HCZ_i(i, A_A, A_MR, A_OR) for i in range(2, 6)])

    return q_rq_H * A_HCZ_hs * f_cT * f_cI

In [ ]:
def get_q_rq_H(region, has_MR_hwh, has_OR_hwh):
    """ 単位面積当たりの必要暖房能力 (W/m2)

    :param region: 省エネルギー地域区分
    :type region: int
    :param has_MR_hwh: 温水暖房の放熱器を主たる居室に設置する場合はTrue
    :type has_MR_hwh: bool
    :param has_OR_hwh: 温水暖房の放熱器をその他の居室に設置する場合はTrue
    :type has_OR_hwh: bool
    :return: 単位面積当たりの必要暖房能力 (W/m2)
    :rtype: float
    """

    if has_MR_hwh and has_OR_hwh:
        return get_table_h_3()[region - 1][0]
    elif has_MR_hwh:
        return get_table_h_3()[region - 1][1]
    elif has_OR_hwh:
        return get_table_h_3()[region - 1][2]
    else:
        raise ValueError('温水暖房の放熱器を主たる居室にもその他の居室にも設置しない場合の単位面積当たりの必要暖房能力は定義されません。')


外気温度補正係数$f_{cT}$は、1.05の値とする。

The outside air temperature correction coefficient $ f_ {cT} $ has a value of 1.05.

In [ ]:
def get_f_cT():
    """ 外気温度補正係数

    :return: 外気温度補正係数
    :rtype: float
    """

    return 1.05


In [ ]:
def get_f_cI(mode_MR, mode_OR, has_MR_hwh, has_OR_hwh):
    """ 間歇運転能力補正係数

    :param mode_MR: 主たる居室の運転方法 (連続運転|間歇運転)
    :type mode_MR: str
    :param mode_OR: その他の居室の運転方法 (連続運転|間歇運転)
    :type mode_OR: str
    :param has_MR_hwh: 温水暖房の放熱器を主たる居室に設置する場合はTrue
    :type has_MR_hwh: bool
    :param has_OR_hwh: 温水暖房の放熱器をその他の居室に設置する場合はTrue
    :type has_OR_hwh: bool
    :return: 間歇運転能力補正係数
    :rtype: float
    """
    def normalize_mode(s):
        if s == 'は':
            return '間歇運転'
        if s == 'ろ':
            return '連続運転'
        return s

    mode_MR = normalize_mode(mode_MR)
    mode_OR = normalize_mode(mode_OR)

    if has_MR_hwh and has_OR_hwh:
        if mode_MR == '連続運転':
            if mode_OR == '連続運転':
                return get_table_h_4()[0]
            elif mode_OR == '間歇運転':
                return get_table_h_4()[1]
            else:
                raise ValueError(mode_OR)
        elif mode_MR == '間歇運転':
            if mode_OR == '連続運転':
                return get_table_h_4()[2]
            elif mode_OR == '間歇運転':
                return get_table_h_4()[3]
            else:
                raise ValueError(mode_MR)
        else:
            raise ValueError(mode_MR)
    elif has_MR_hwh:
        if mode_MR == '連続運転':
            return get_table_h_4()[4]
        elif mode_MR == '間歇運転':
            return get_table_h_4()[5]
        else:
            raise ValueError(mode_MR)
    elif has_OR_hwh:
        if mode_OR == '連続運転':
            return get_table_h_4()[6]
        elif mode_OR == '間歇運転':
            return get_table_h_4()[7]
        else:
            raise ValueError(mode_OR)

<div style="text-align: center;font-weight: bold;">表　単位面積当たりの必要暖房能力(W/m<sup>2</sup>)<br>Table: Required heating capacity per unit area</div>

|地域の区分<br>Segmentation of area|（い）温水暖房の放熱器を主たる居室及びその他の居室に設置する場合<br>(I) When installing a radiator of hot water heating in the main room and other rooms|（ろ）温水暖房の放熱器を主たる居室に設置しその他の居室に設置しない場合<br>(RO) When the radiator of hot water heating is installed in the main room and not installed in other rooms|（は）温水暖房の放熱器を主たる居室に設置せずその他の居室に設置する場合<br>(HA) When installing in a room other than the main room without installing a radiator for hot water heating|
|-|-|-|-|
|1|90.02|139.26|62.28|
|2|77.81|120.65|53.26|
|3|73.86|111.32|53.81|
|4|77.74|118.98|55.41|
|5|83.24|126.56|59.43|
|6|69.76|106.48|49.93|
|7|74.66|112.91|53.48|
|8|－|－|－|


In [ ]:
def get_table_h_3():
    """ 表 H.3 単位面積当たりの必要暖房能力

    :return: 表 H.3 単位面積当たりの必要暖房能力
    :rtype: list
    """
    table_h_3 = [
        (90.02, 139.26, 62.28),
        (77.81, 120.65, 53.26),
        (73.86, 111.32, 53.81),
        (77.74, 118.98, 55.41),
        (83.24, 126.56, 59.43),
        (69.76, 106.48, 49.94),
        (74.66, 112.91, 53.48)
    ]
    return table_h_3


<div style="text-align: center;font-weight: bold;">表　間歇運転能力補正係数<br>Table: Intermittent driving ability correction factor</div>

|主たる居室の運転方法<br>How to operate the main room|その他の居室の運転方法<br>How to operate other rooms|間歇運転能力補正係数<br>Intermittent driving ability correction factor|
|-|-|-|
|連続運転<br>Continuous operation|連続運転<br>Continuous operation|1.0|
|連続運転<br>Continuous operation|間歇運転<br>Intermittent operation|1.0|
|間歇運転<br>Intermittent operation|連続運転<br>Continuous operation|2.25|
|間歇運転<br>Intermittent operation|間歇運転<br>Intermittent operation|2.25|
|連続運転<br>Continuous operation|放熱器を設置しない<sup>※</sup><br>Do not install a radiator|1.0|
|間歇運転<br>Intermittent operation|放熱器を設置しない<sup>※</sup><br>Do not install a radiator|3.03|
|放熱器を設置しない<br>Do not install a radiator|連続運転<br>Continuous operation|1.0|
|放熱器を設置しない<br>Do not install a radiator|間歇運転<br>Intermittent operation|1.62|


※ その他の居室が無い場合も含む。 

※ Including cases where there are no other rooms.

In [ ]:
def get_table_h_4():
    """ 表 H.4 間歇運転能力補正係数

    :return: 表 H.4 間歇運転能力補正係数
    :rtype: list
    """
    table_h_4 = [
        1.0,
        1.0,
        2.25,
        2.25,
        1.0,
        3.03,
        1.0,
        1.62
    ]
    return table_h_4
